In [2]:
import cv2
import numpy as np
import math
import glob
from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans
from matplotlib import pyplot as plt
from IPython.display import Image, display

In [ ]:
img = cv2.cvtColor(cv2.imread("../reduced_dataset/men/4/4_men (141).JPG"),cv2.COLOR_BGR2YCrCb)
# plt.imshow(img)
# img = cv2.resize(src=img,dsize=(int(img.shape[1]/8),int(img.shape[0]/8)))
# img = cv2.bilateralFilter(img,d=5,sigmaColor=10,sigmaSpace=10)
img_vector = img.reshape([-1,3])
# img_vector = img_vector[:,1:3]
img_vector[:,0] = 100
# img_vector[:,2] = img_vector[:,2] * 12
# img_vector[:,1] = img_vector[:,1] * 6
# img_vector_YExcluded = img_vector[:,1:]
bw = estimate_bandwidth(X=img_vector,quantile=0.1,n_samples=500)
# bw
print(bw)
ms = MeanShift(bandwidth=bw,bin_seeding=True)
ms.fit(img_vector)
print(ms.cluster_centers_.shape)
# img

# img_vector[:,1:3] = img_vector[:,1:3] / 5
result_image = ms.cluster_centers_.astype(dtype=np.uint8)[ms.labels_].reshape((img.shape[0],img.shape[1],3))
# result_image = ms.cluster_centers_.astype(dtype=np.uint8)[ms.labels_].reshape((img.shape[0],img.shape[1],2))
# result_image = cv2.cvtColor(result_image,cv2.COLOR_YCrCb2BGR)
# result_image = cv2.inRange(result_image,(0,137,77),(255,163,140))
# result_image[:,2] = result_image[:,2] / 12
# result_image[:,1] = result_image[:,1] / 6
# result_image = np.concatenate(np.ones((img.shape[0], img.shape[1],1),dtype=np.uint8)*128,result_image)
cv2.imwrite("compressed.JPG",result_image)
# plt.imshow(result_image)
# print(ms.cluster_centers_[ms.labels_])
# fig = plt.figure(figsize=(12,10))
# ax = fig.add_subplot(111,projection='3d')
# ax.scatter(img[0,:,0],img[0,:,1],img[0,:,2])
# plt.show()

In [ ]:
img = cv2.cvtColor(cv2.imread("../reduced_dataset/men/4/4_men (141).JPG"),cv2.COLOR_BGR2YCrCb)
bg_mask = np.bitwise_not(cv2.inRange(img,(0,137,77),(255,163,120))).reshape((-1))
fg_mask = cv2.inRange(img,(0,137,77),(255,163,120)).reshape((-1))
# print(bg_mask.shape)
# print(img.reshape((-1,3)).shape)
# print(img.reshape((-1,3))[bg_mask == 255].shape)
bg_mean = np.mean(img.reshape((-1,3))[bg_mask == 255], axis = 0)
fg_mean = np.mean(img.reshape((-1,3))[fg_mask != 0], axis = 0)
print(bg_mean)
print(fg_mean)

# plt.imshow(img)
# img = cv2.resize(src=img,dsize=(int(img.shape[1]/8),int(img.shape[0]/8)))
# img = cv2.bilateralFilter(img,d=5,sigmaColor=10,sigmaSpace=10)
img_vector = img.reshape([-1,3])
# img_vector[:,0] = 100
divisor = 20
img_vector[:,1] = (np.tanh((img_vector[:,1] - 137)/divisor) - np.tanh((img_vector[:,1] - 163)/divisor)) * img_vector[:,1]/2
img_vector[:,2] = (np.tanh((img_vector[:,2] - 77)/divisor) - np.tanh((img_vector[:,2] - 120)/divisor)) * img_vector[:,2]/2
# (0,137,77),(255,163,120)
# cv2.imwrite("compressed.JPG",img_vector.reshape((img.shape[0],img.shape[1],3)))


# km = KMeans(n_clusters=3)
km = KMeans(n_clusters=2, init=np.array([[bg_mean[0],0,0], fg_mean]), max_iter=3)
# km = KMeans(n_clusters=3, init=np.array([bg_mean,[30, 149, 96],[200, 149, 96]]), max_iter=3)
km.fit(img_vector)

result_image = km.cluster_centers_.astype(dtype=np.uint8)[km.labels_].reshape((img.shape[0],img.shape[1],3))
# result_image = cv2.cvtColor(result_image,cv2.COLOR_YCrCb2BGR)
cv2.imwrite("compressed.JPG",result_image)

In [ ]:
x=0
for filename in sorted(glob.glob('data_set/men/0/*.JPG')):
    img = cv2.imread(filename) 
    img = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
    img_masked = cv2.inRange(img,(0,137,77),(255,163,120))
    cv2.imwrite("masks/"+str(x)+".jpg",img_masked)
    x+=1
# plt.imshow(img_masked)
# np.array((0,137,77),dtype=np.uint8)
# img_masked

In [22]:
img = cv2.imread("reduced_dataset/men/3/3_men (64).JPG")
# img = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
# img_masked = cv2.inRange(img,(0,136,77),(255,163,140))
img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
img_masked_shadow = np.bitwise_not(cv2.inRange(img,(0,0,80),(255,55,178)))
img_masked_hand = cv2.inRange(img,(5,35,30),(25,255,255))
img_masked = np.bitwise_and(img_masked_shadow,img_masked_hand)
cv2.imwrite("masked.JPG",img_masked)

True

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
img = cv.imread('D:/pattern/research/dataset_sample/men/3/3_men (2).JPG', cv.IMREAD_GRAYSCALE)
assert img is not None, "file could not be read, check with os.path.exists()"
# global thresholding
ret1,th1 = cv.threshold(img,127,255,cv.THRESH_BINARY)
# Otsu's thresholding
ret2,th2 = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# Otsu's thresholding after Gaussian filtering
blur = cv.GaussianBlur(img,(5,5),0)
ret3,th3 = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# plot all the images and their histograms
images = [img, 0, th1,
          img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
          'Original Noisy Image','Histogram',"Otsu's Thresholding",
          'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]
for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
import cv2
import mediapipe as mp
import time

# imgWidth = 1280
# imgHeight = 720
# cap = cv2.VideoCapture(0)
# cap.set(3, imgWidth)
# cap.set(4, imgHeight)
mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# pTime = 0
# cTime = 0
filename = 'data_set/men/3/3_men (5).JPG'
img = cv2.imread(filename)
img= cv2.flip(img,1)
img_b = np.empty(img.shape)
img_b.fill(0)
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = hands.process(imgRGB)
#print(results.multi_hand_landmarks)
if results.multi_hand_landmarks:
    for handLms in results.multi_hand_landmarks: 
    #handLMs are 21 points. so we need conection too-->mpHands.HAND_CONNECTIONS
        for id, lm in enumerate(handLms.landmark):
            #print(id, lm)
            #lm = x,y cordinate of each landmark in float numbers. lm.x, lm.y methods
            #So, need to covert in integer
            h, w, c =img.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            #print(id, cx, cy)
            # if id == 4: #(To draw 4th point)
            #cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        mpDraw.draw_landmarks(img_b, handLms, mpHands.HAND_CONNECTIONS) #drawing points and lines(=handconections)


cv2.imwrite('masked.JPG', img_b)

In [ ]:
# # Parameters
# blur = 21
# canny_low = 15
# canny_high = 150
# min_area = 0.0005
# max_area = 0.95
# dilate_iter = 10
# erode_iter = 10
# mask_color = (0.0,0.0,0.0)
# filename = 'D:/pattern/research/dataset_sample/men/3/3_men (1).JPG'
# # Convert image to grayscale        
# img = cv2.imread(filename) 
# image_gray = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
# # Apply Canny Edge Dection
# edges = cv2.Canny(image_gray, canny_low, canny_high)

# edges = cv2.dilate(edges, None)
# edges = cv2.erode(edges, None)

# contour_info = []

# c 

# for c in cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)[0]:

#     contour_info.append((
#         c,
#         cv2.contourArea(c),
#     ))
    

# # get the contours and their areas


# # Set up mask with a matrix of 0's
# mask = np.zeros(edges.shape, dtype = np.uint8)


# # Go through and find relevant contours and apply to mask
# for contour in contour_info:
# # Instead of worrying about all the smaller contours, if the area is smaller than the min, the loop will break
#     if contour[1] > min_area and contour[1] < max_area:
#         # Add contour to mask
#         mask = cv2.fillConvexPoly(mask, contour[0], (255))

# # use dilate, erode, and blur to smooth out the mask
# # use dilate, erode, and blur to smooth out the mask
# mask = cv2.dilate(mask, None)
# mask = cv2.erode(mask, None)
# mask = cv2.GaussianBlur(mask, (blur, blur), 0)

# # Ensures data types match up
# mask_stack = mask.astype('float32') / 255.0           
# img = img.astype('float32') / 255.0

# # Creates a 3 channel image

# # Multiplies mask_stack with image to get masked image




# print(mask_stack.shape)
# print(img.shape)

 
# # Blend the image and the mask
# masked = (mask_stack * img) + ((1-mask_stack) * mask_color)
# masked = (masked * 255).astype('uint8')
# cv2.imwrite("masked.jpg", masked)




In [ ]:
# Required modules
import cv2
import numpy as np
import matplotlib.pyplot as plt





min_YCrCb = np.array([0,133,77],np.uint8)
max_YCrCb = np.array([235,173,140],np.uint8)

min_HSV = np.array([0, 58, 30], dtype = "uint8")
max_HSV = np.array([33, 255, 255], dtype = "uint8")

# Get pointer to video frames from primary device
image = cv2.imread("D:/pattern/research/dataset_sample/men/3/3_men (2).JPG")

imageYCrCb = cv2.cvtColor(image,cv2.COLOR_BGR2YCR_CB)
skinRegionYCrCb = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)

skinYCrCb = cv2.bitwise_and(image, image, mask = skinRegionYCrCb)


imageHSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
skinRegionHSV = cv2.inRange(imageHSV, min_HSV, max_HSV)

skinHSV = cv2.bitwise_and(image, image, mask = skinRegionHSV)



# ret1,th1 = cv.threshold(img,127,255,cv.THRESH_BINARY)
# # Otsu's thresholding
# ret2,th2 = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# # Otsu's thresholding after Gaussian filtering
blur = cv.GaussianBlur(img,(5,5),0)
ret3,th3 = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

# # plot all the images and their histograms

cv2.imwrite("masked.jpg", th3)
cv2.imwrite("masked2.jpg", np.hstack([image,skinYCrCb]))
cv2.imwrite("masked3.jpg", np.hstack([image,skinHSV]))

print (th3.shape)
print (image.shape)
print (skinYCrCb.shape)
print (skinHSV.shape)










cv2.imwrite("masked2.jpg", np.hstack([image,skinYCrCb]))

In [ ]:
# def convolve(B, r):
#     D = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(r,r))
#     cv2.filter2D(B, -1, D, B)
#     return B

# #Loading the image and converting to HSV
# image = cv2.imread("D:/pattern/research/dataset_sample/men/3/3_men (2).JPG")
# image_hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
# model_hsv = image_hsv[225:275,625:675] # Select ROI

# #Get the model histogram M
# M = cv2.calcHist([model_hsv], channels=[0, 1], mask=None, 
#                   histSize=[80, 256], ranges=[0, 180, 0, 256] )

# #Backprojection of our original image using the model histogram M
# B = cv2.calcBackProject([image_hsv], channels=[0,1], hist=M, 
#                          ranges=[0,180,0,256], scale=1)

# B = convolve(B, r=5)

# #Threshold to clean the image and merging to three-channels
# _, thresh = cv2.threshold(B, 0, 255, cv2.THRESH_BINARY)
# cv2.imwrite("masked.jpg",cv2.cvtColor(model_hsv,cv2.COLOR_HSV2RGB))
# cv2.imwrite("masked2.jpg",cv2.bitwise_and(image,image, mask = thresh))

In [ ]:
# Required modules
import cv2
import numpy as np
import matplotlib.pyplot as plt

min_HSV = np.array([0, 58, 30], dtype = "uint8")
max_HSV = np.array([33, 255, 255], dtype = "uint8")
# Get pointer to video frames from primary device
image = cv2.imread("D:/pattern/research/dataset_sample/men/3/3_men (2).JPG")
imageHSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
skinRegionHSV = cv2.inRange(imageHSV, min_HSV, max_HSV)

skinHSV = cv2.bitwise_and(image, image, mask = skinRegionHSV)

cv2.imwrite("masked.jpg", np.hstack([image, skinHSV]))